In [1]:
data_path = 'Weight/'
model_weights = 'weight_classification.h5'

batch_size = 32
valid_size = 16
color_mode = 'rgb'

width = 299
height = 299

target_size = (width, height)
input_shape = (width, height, 3)

zoom_range = 0.3
shear_range = 0.3
shift_range = 0.3
rotation_range = 30

dense_1 = 512
dense_2 = 256
dense_3 = 64
num_classes = 5

epochs = 50
rate = 0.2

verbose = 1

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
############################################################################################

def preprocessing_function(img):
    img = tf.keras.applications.xception.preprocess_input(img)
    return img

def image_data_generator():
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rotation_range = rotation_range,
                                                                shear_range = shear_range,
                                                                zoom_range = zoom_range,
                                                                width_shift_range=shift_range,
                                                                height_shift_range=shift_range,
                                                                horizontal_flip = True,
                                                                preprocessing_function=preprocessing_function
                                                                )

    data_generator = train_datagen.flow_from_directory(
                                    data_path,
                                    target_size = target_size,
                                    color_mode = color_mode,
                                    batch_size = batch_size,
                                    class_mode = 'categorical',
                                    shuffle = True
                                    )

    return data_generator

In [3]:
data_generator = image_data_generator()
data_generator.class_indices

Found 1176 images belonging to 5 classes.


{'183lbs-278lbs_Body': 0,
 '259lbs-548lbs_Body': 1,
 '93lbs-177lbs_Body': 2,
 'Above 498lbs_Body': 3,
 'Unknown': 4}

In [4]:
class CattleWeightClassification(object):
    def __init__(self):
        self.data_generator = data_generator
        self.train_step = self.data_generator.samples // batch_size

        self.accuracy = tf.keras.metrics.CategoricalAccuracy()
        self.recall = tf.keras.metrics.Recall()
        self.precision = tf.keras.metrics.Precision()

        self.id2weight = {v:k for k, v in self.data_generator.class_indices.items()}

    def classifier(self, x):
        if not self.trainable:
            x = Dense(dense_1)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

            x = Dense(dense_2)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

        x = Dense(dense_3)(x)
        x = BatchNormalization()(x)
        x = relu(x)
        x = Dropout(rate)(x)
        return x

    def model_conversion(self, trainable):
        functional_model = tf.keras.applications.Xception(weights="imagenet")
        functional_model.trainable = trainable

        self.trainable = trainable

        inputs = functional_model.input

        x = functional_model.layers[-2].output
        x = self.classifier(x)
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(
                inputs=inputs,
                outputs=outputs
                    )
                    
        self.model = model
        self.model.summary()

    def train(self):
        callback = tf.keras.callbacks.EarlyStopping(
                                                monitor='loss', 
                                                patience=5
                                                    )

        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                          )
        self.model.fit(
                    self.data_generator,
                    steps_per_epoch= self.train_step,
                    epochs=epochs,
                    verbose=verbose
                        )

    def save_model(self):
        self.model.save(model_weights)

    def load_model(self):
        self.model = load_model(model_weights)
        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                         )

    def predict(self, x):
        x = cv.resize(x, target_size)
        x = preprocessing_function(x)
        x = np.expand_dims(x, axis=0)
        P = self.model.predict(x)
        idx = np.argmax(P)
        return self.id2weight[idx]
        
    def process(self):
        if not os.path.exists(model_weights):
            self.model_conversion(False)
            self.train()
            self.save_model()
        else:
            self.load_model()

In [5]:
model = CattleWeightClassification()
model.process()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 37, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 37, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 19, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 19, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 19, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 19, 19, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 19, 19, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 19, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 19, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 19, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

                                                                                                  
 batch_normalization_6 (BatchNo  (None, 64)          256         ['dense_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.nn.relu_2 (TFOpLambda)      (None, 64)           0           ['batch_normalization_6[0][0]']  
                                                                                                  
 dropout_2 (Dropout)            (None, 64)           0           ['tf.nn.relu_2[0][0]']           
                                                                                                  
 dense_3 (Dense)                (None, 5)            325         ['dropout_2[0][0]']              
                                                                                                  
Total para

# **Inference**

In [6]:
img_path = 'Weight/93lbs-177lbs_Body/bhnhn.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'93lbs-177lbs_Body'

In [7]:
img_path = 'Weight/Unknown/agalychnis-callidryas_63_d690b0a9.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Unknown'

In [8]:
img_path = 'Weight/Unknown/istockphoto-1365223878-170667a.jpg'
img = cv.imread(img_path)
P = model.predict(img)
P

'Unknown'

In [9]:
img_path = 'Weight/183lbs-278lbs_Body/2E.jpg'
img = cv.imread(img_path)
P = model.predict(img)  
P

'259lbs-548lbs_Body'

In [10]:
img_path = 'Weight/259lbs-548lbs_Body/Ayrshirecattle78_c.jpg'
img = cv.imread(img_path)
P = model.predict(img) 
P

'259lbs-548lbs_Body'